In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

# Scrape data from Brilliant Earth

In [2]:
colNames = ['Stock Number','Gemstone','Origin','Price','Carat Weight','Shape','Cut','Color',
                               'Clarity','Measurements','Table','Depth','Symmetry','Polish', 'Girdle',
                              'Culet','Fluorescence']
data = []
nCols = len(colNames)

In [3]:
url = 'https://www.brilliantearth.com/loose-diamonds/view_detail/'

In [4]:
def processDiamond(soup):
    soupDL = soup.findAll('dl')
    #find where the data table starts in the soupDL list
    starting = [i for i in range(len(soupDL)) if "Stock Number" in str(soupDL[i])][0]
    #iterate through every position from starting position through end of soupDL list
    #check whether the field matches name of column in our data table
    diamondDict = {}
    for pos in range(starting, len(soupDL)):
        s = str(soupDL[pos])
        if ('Stock Number' in s) or ('Gemstone' in s):
            fieldName = s[s.find('<dt>')+4:s.find('</dt>')-1]
            val = s[s.find('<dd>')+4:s.find('</dd>')]
            diamondDict[fieldName] = val
        elif 'Origin' in s:
            fieldName = 'Origin'
            val = s[s.find('<dd>')+4:s.find('</dd>')]
            diamondDict[fieldName] = val
        elif ('Girdle' in s) or ('Polish' in s):
            fieldName = s[s.find('</a>')+4:s.find('</dt>')-10]
            val = s[s.find('<dd>')+4:s.find('</dd>')]
            diamondDict[fieldName] = val
        else:
            fieldName = s[s.find('</a>')+4:s.find('</dt>')-1]
            if fieldName in colNames:
                val = s[s.find('<dd>')+4:s.find('</dd>')]
                diamondDict[fieldName] = val
    return diamondDict

In [5]:
def addDiamond(diamondId):
    num_retries = 0
    got_response = False
    while True:
        try:
            link = requests.get(f"{url}{diamondId}/", timeout=5)
            got_response = True
        except Exception as exception:
            if num_retries < 5:
                num_retries += 1
            time.sleep(4**num_retries)
        if got_response:
            break
    soup = BeautifulSoup(link.content,'html.parser')
    if 'error404' not in str(soup):        
        diamondDict = processDiamond(soup)
        data.append(diamondDict)
    time.sleep(0.3)

In [6]:
diamondList = range(6621210,6650000)
for i in tqdm(diamondList):
    addDiamond(i)

100%|██████████| 28790/28790 [16:24:57<00:00,  1.09s/it]


In [8]:
df = pd.DataFrame(data)

In [9]:
df.tail()

,Carat Weight,Clarity,Color,Cut,Depth,Fluorescence,Gemstone,Girdle,Measurements,Origin,Polish,Price,Shape,Stock Number,Symmetry,Table
16217,0.32,VVS2,E,Super Ideal,61.6%,None,"Natural, untreated diamond","Medium - Slightly Thick, Faceted",4.38mm x 4.36mm x 2.69mm,Botswana Sort,Excellent,"$1,030",Round,2720882A,Excellent,59.0%
16218,0.34,SI1,D,Ideal,62.4%,Faint,"Natural, untreated diamond","Medium - Slightly Thick, Faceted",4.47mm x 4.45mm x 2.79mm,Canada,Excellent,$950,Round,2720880A,Very Good,57.0%
16219,0.80,VVS1,H,Super Ideal,62.2%,None,"Natural, untreated diamond","Thin - Slightly Thick, Faceted",5.96mm x 5.91mm x 3.69mm,Botswana Sort,Excellent,"$4,170",Round,2976751A,Excellent,58.0%
16220,0.30,SI1,E,Very Good,61.8%,None,"Natural, untreated diamond","Medium - Thick, Faceted",4.32mm x 4.29mm x 2.66mm,Botswana Sort,Excellent,$840,Round,2721027A,Excellent,60.0%
16221,0.90,VS1,G,Very Good,64.6%,None,"Natural, untreated diamond","Thin - Thick, Faceted",6.06mm x 6.02mm x 3.91mm,Botswana Sort,Excellent,"$4,680",Round,2963872A,Very Good,55.0%


In [10]:
df.to_csv('/mnt/c/Users/tmngu/Documents/diamond/data6650000.csv')

In [11]:
df.to_csv('data6650000.csv')